In [109]:
import psycopg2
import numpy as np

In [9]:
conn = psycopg2.connect(database="curtis",
                        user="curtis",
                        password="apassword",
                        host="localhost")
conn.autocommit = True
cur = conn.cursor()

In [91]:
query = """SELECT specs ->> 'url' AS url,
                  specs ->> 'color' AS color,
                  jsonb_array_elements_text(specs -> 'memory') AS memory,
                  specs ->> 'screen' AS screen,
                  jsonb_array_elements_text(specs -> 'storage') AS storage,
                  (specs ->> 'price')::float AS price
            FROM mac_refurb
            WHERE LOWER(specs ->> 'url') LIKE '%macbook-pro%' AND
                  specs ->> 'date' LIKE '%2018'
"""

In [92]:
cur.execute(query)

In [93]:
keys = ['url', 'color', 'memory', 'screen', 'storage', 'price']
raw = [dict(zip(keys, line)) for line in cur]

In [94]:
raw[0]

{'url': 'https://www.apple.com/shop/product/FR9U2LL/A/Refurbished-133-inch-MacBook-Pro-23GHz-quad-core-Intel-Core-i5-with-Retina-display-Silver',
 'color': 'silver',
 'memory': '8gb of 2133mhz lpddr3 onboard memory',
 'screen': '13.3-inch (diagonal) led-backlit display with ips technology; 2560-by-1600 native resolution at 227 pixels per inch',
 'storage': '256gb ssd 1',
 'price': 1489.0}

In [95]:
def clean_screen(item):
    if '13.3' in item:
        return '13inch'
    elif '15.4' in item:
        return '15inch'
    else:
        return "N/A"

In [96]:
def clean_storage(item):
    if '256gb' in item:
        return '256GB'
    elif '512gb' in item:
        return '512MB'
    elif '1tb' in item:
        return '1TB'
    elif '2tb' in item:
        return '2TB'
    elif '4tb' in item:
        return '4TB'
    else:
        return "N/A"

In [122]:
clean = []
for line in raw:
    row = {
        'color': line['color'],
        'screen': clean_screen(line['screen']),
        'storage': clean_storage(line['storage']),
        'price': line['price']
    }
    clean.append(row)

In [123]:
clean[0]

{'color': 'silver', 'screen': '13inch', 'storage': '256GB', 'price': 1489.0}

In [124]:
def one_hot(data, key):
    
    values = set([line[key] for line in data])
    
    for line in data:
        for value in values:
            if line[key] == value:
                line[key + "_" + value] = 1
            else:
                line[key + "_" + value] = 0
                
        del(line[key])
    
    return data

In [125]:
new = one_hot(clean, 'screen')
new = one_hot(new, 'color')
new = one_hot(new, 'storage')

In [126]:
new[0]

{'price': 1489.0,
 'screen_15inch': 0,
 'screen_13inch': 1,
 'color_space-gray': 0,
 'color_silver': 1,
 'storage_1TB': 0,
 'storage_4TB': 0,
 'storage_2TB': 0,
 'storage_512MB': 0,
 'storage_256GB': 1}

In [138]:
x = []
y = []
for line in new:
    nums = []
    for key in line.keys():
        if key == 'price':
            y.append(line[key])
        else:
            nums.append(line[key])
    x.append(np.array(nums))
    
x = np.array(x)
y = np.array(y)

In [139]:
y

array([1489., 1489., 1659., 1659., 1739., 1909., 1999., 1999., 1999.,
       1999., 2169., 2169., 2169., 2249., 2289., 2339., 2289., 2339.,
       2339., 2419., 2419., 2509., 2509., 2509., 2509., 2509., 2509.,
       2549., 2629., 2629., 2679., 2759., 2759., 2849., 2849., 2849.,
       2849., 2849., 2849., 2849., 2849., 2889., 2929., 2889., 2889.,
       2929., 2889., 2929., 2929., 2969., 2969., 3099., 3139., 3139.,
       3189., 3189., 3189., 3229., 3229., 3269., 3399., 3439., 3439.,
       3439., 3479., 3479., 3529., 3529., 3699., 3739., 3739., 4039.,
       5009., 5009., 5269., 5269., 5569., 5569., 2079., 2509., 2509.,
       2759., 3359., 3399., 3439., 2169., 1659., 1659., 4669., 2419.,
       1829., 1829., 2549., 1909., 1909., 2079., 2249., 3269., 3359.,
       1999., 2339., 2509., 2589., 2589., 2589., 2629., 2629., 2849.,
       2849., 2249., 2249., 2339., 2339., 3099., 3189., 2459., 2079.,
       2929., 2679., 2679., 2679., 2419., 2759., 2589., 2799., 2799.,
       2679., 2799.,

In [140]:
x

array([[0, 1, 0, ..., 0, 0, 1],
       [0, 1, 1, ..., 0, 0, 1],
       [0, 1, 1, ..., 0, 1, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0]])

In [152]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [147]:
fit = LinearRegression().fit(x, y)

In [148]:
fit.score(x, y)

0.8214036939258825

In [150]:
y_pred = fit.predict(x)

In [153]:
mean_squared_error(y, y_pred)

104807.75776397515

In [154]:
r2_score(y, y_pred)

0.8214036939258825

In [155]:
import pickle

In [157]:
s = pickle.dumps(fit)

In [158]:
fit2 = pickle.loads(s)

In [159]:
fit2.predict(x[:10])

array([1616., 1648., 1920., 1888., 1648., 1648., 2208., 2408., 2384.,
       2176.])